In [32]:
print 'starting'

from robobrowser import RoboBrowser
from textblob import TextBlob
from pattern.es import tag
urls = 'http://www.elmostrador.cl/','http://www.elmostrador.cl/noticias/pais','http://www.elmostrador.cl/noticias/mundo','http://www.elmostrador.cl/noticias/opinion'
urls = urls[:1]
from pattern.vector import KNN, count
from collections import Counter, defaultdict
import indicoio
indicoio.config.api_key = '37958de57fa5f9d8d4bbec0c0902fd1e'



browser = RoboBrowser(history=True, parser='html5lib')
palabras=Counter()
palabras_sentimiento = defaultdict(list)
articulos = []
for url in urls:
    print url
    browser.open(url)
    articulos = browser.select('h3 a') + browser.select('h2 a')
    for articulo in articulos:
        if '2016' not in articulo.attrs['href']:
            continue
        browser.follow_link(articulo)
        texto = browser.select('.cuerpo-noticia')
        if texto:
            texto = texto[0].text
        else:
            continue        
        titulo = browser.select('.datos-noticias h2')
        if titulo:
            titulo = titulo[0].text
        else:
            continue
        v = tag(texto)
        v = [word for word, pos in v if pos == 'JJ'] # JJ = adjective
        palabras_articulo = Counter(v)
        palabras.update(palabras_articulo)
        if palabras_articulo:
            sentiment = indicoio.sentiment(texto, language='spanish')
            print u"%s" % titulo
            print palabras_articulo.most_common(10)
            print sentiment
            print '----'
            for i,c in palabras_articulo.items():
                palabras_sentimiento[i].append(sentiment)

        print palabras_sentimiento.items()[:10]

starting
http://www.elmostrador.cl/
La receta “fascista” de Starace para hacer negocios hace olas entre los que se opusieron a reorganización de Enersis
[(u'ejecutivo', 3), (u'fascista', 2), (u'italianos', 2), (u'minoritarios', 2), (u'irregulares', 1), (u'solo', 1), (u'Primer', 1), (u'digitales', 1), (u'pol\xedticos', 1), (u'pol\xedticas', 1)]
0.75137231785
----
[(u'irregulares', [0.75137231785009]), (u'solo', [0.75137231785009]), (u'capaces', [0.75137231785009]), (u'fascista', [0.75137231785009]), (u'digitales', [0.75137231785009]), (u'ejecutivo', [0.75137231785009]), (u'pol\xedticos', [0.75137231785009]), (u'pol\xedticas', [0.75137231785009]), (u'cierta', [0.75137231785009]), (u'italianos', [0.75137231785009])]
Chile, el país más rico en litio del mundo puede quedar al margen del boom de Tesla Motors
[(u'ex', 2), (u'chileno', 2), (u'vecina', 2), (u'el\xe9ctrico', 2), (u'favorable', 1), (u'gubernamental', 1), (u'r\xe1pido', 1), (u'propia', 1), (u'primer', 1), (u'legal', 1)]
0.77695414

In [44]:
puntajes = []
for palabra, sentimientos in palabras_sentimiento.items():
    if sentimiento:
        cantidad = len(sentimientos)
        if cantidad:
            promedio = sum(sentimientos)/cantidad
            puntajes.append([cantidad,promedio,palabra])

In [50]:
puntajes.sort()
puntajes.reverse()
puntajes[:100]

[[13, 0.8008936489318526, u'nuevo'],
 [13, 0.7637912496471717, u'solo'],
 [12, 0.7810163744325233, u'mismo'],
 [9, 0.8538984245908949, u'gran'],
 [9, 0.7300309589862523, u'nueva'],
 [8, 0.7645676212372456, u'primer'],
 [7, 0.762243773509998, u'p\xfablica'],
 [7, 0.7368081000826724, u'pol\xedtico'],
 [6, 0.8676019386089089, u'nacional'],
 [6, 0.8485011542396803, u'social'],
 [6, 0.8443066673045537, u'propia'],
 [6, 0.8381253589270466, u'primera'],
 [6, 0.8254775924244656, u'sociales'],
 [6, 0.7680430180334068, u'ex'],
 [6, 0.7678274559136042, u'mejor'],
 [6, 0.7518341260987755, u'\xfaltimo'],
 [6, 0.7324529488946391, u'tercera'],
 [6, 0.7234447999195474, u'f\xe1cil'],
 [6, 0.675155843146304, u'anterior'],
 [5, 0.8728231132700568, u'importante'],
 [5, 0.8660066642791809, u'nuevas'],
 [5, 0.8364969143499437, u'mayor'],
 [5, 0.8311443509743356, u'\xfaltimos'],
 [5, 0.7972021967139062, u'estatal'],
 [5, 0.7960148143653307, u'misma'],
 [5, 0.743586849609852, u'pol\xedtica'],
 [5, 0.730487391

In [64]:
import plotly.plotly as py
import plotly.graph_objs as go
import plotly.plotly as py
import numpy as np
py.sign_in('RodrigoCulagovski', 'hjhyib8mym')

x,y,text = zip(*puntajes)
trace = go.Scatter(
    x=x,
    y=y,
    text=text,
    mode='markers',
    marker=dict(size=12,
                line=dict(width=1)
               ),
    name='Adjetivos',
    )
data = [trace]
layout = go.Layout(
    title='Frecuencia de palabras',
    hovermode='closest',
    xaxis=dict(
        title='Frecuencia de palabras',
        ticklen=5,
        zeroline=False,
        gridwidth=2,
    ),
    yaxis=dict(
        title='Análisis de Sentimiento',
        ticklen=5,
        gridwidth=2,
    ),
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='palabras')